# Prep

In [141]:
import pandas as pd
import numpy as np
import pprint
import re
import os
import psycopg2
import janitor

from datetime import datetime
from dotenv import load_dotenv
from psycopg2 import sql, extras

In [142]:
WORKOUTS_FILE = r'C:\Users\User\Documents\GitHub\ap_workouts\b1-bronze\workouts.csv'
PLANS_FILE = r'C:\Users\User\Documents\GitHub\ap_workouts\b1-bronze\plans.csv'
CUSTOMEXERCISES_FILE = r'C:\Users\User\Documents\GitHub\ap_workouts\b1-bronze\custom_exercises.csv'
SILVER_FOLDER = r'C:\Users\User\Documents\GitHub\ap_workouts\b2-silver'
GOLD_FOLDER = r'C:\Users\User\Documents\GitHub\ap_workouts\b3-gold'

TEST_STANDALONEWORKOUTS_FILE = r'C:\Users\User\Documents\GitHub\ap_workouts\tests\test_standaloneworkout.csv'

## Auth

In [143]:
load_dotenv(r"C:\\Users\\User\\Documents\\GitHub\\notion_api\\postgres_auth.env")

db_name = os.getenv("dbname")
db_user = os.getenv("user")
db_password = os.getenv("password")
db_host = os.getenv("host")
db_port = os.getenv("port")

## Functions

### Extract

### Transform

#### Plans

In [144]:
def parse_plans(file_path):
    # Read the raw file
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()
    
    # Clean lines and remove empty ones
    lines = [line.strip() for line in lines if line.strip()]
    
    parsed_data = []
    current_program = None
    current_day = None
    current_day_number = None
    
    for line in lines:
        # Remove quotes and split by semicolon
        parts = [part.strip('"') for part in line.split(';')]
        
        # Check if this is a program header (has date as second part)
        if len(parts) == 2 and re.match(r'\d{4}-\d{2}-\d{2}', parts[1]):
            # Parse program info
            program_info = parts[0]
            start_date = parts[1]
            
            # Extract program name and duration
            if '·' in program_info:
                program_parts = program_info.split('·')
                plan_name = program_parts[0].strip()
                duration_str = program_parts[1].strip()
                
                # Extract number of weeks
                duration_match = re.search(r'(\d+)\s*weeks?', duration_str, re.IGNORECASE)
                duration_weeks = int(duration_match.group(1)) if duration_match else None
            else:
                plan_name = program_info
                duration_weeks = None
            
            current_program = {
                'plan_name': plan_name,
                'duration_weeks': duration_weeks,
                'start_date': start_date
            }
            continue
        
        # Check if this is a day header
        if len(parts) == 1 and parts[0].startswith('Day '):
            day_info = parts[0]
            
            # Extract day number and name
            if '·' in day_info:
                day_parts = day_info.split('·')
                day_number_str = day_parts[0].strip()
                day_name = day_parts[1].strip()
                
                # Extract day number
                day_match = re.search(r'Day\s+(\d+)', day_number_str)
                current_day_number = int(day_match.group(1)) if day_match else None
                current_day = day_name
            continue
        
        # Check if this is an exercise line
        if len(parts) >= 3 and parts[0] and re.match(r'^\d+\.', parts[0]):
            # Parse exercise info
            exercise_info = parts[0]
            goal_sets = parts[1] if len(parts) > 1 else None
            goal_quantity_info = parts[2] if len(parts) > 2 else None
            
            # Extract exercise number, name, and equipment
            exercise_match = re.match(r'^(\d+)\.\s*(.+?)(?:\s+·\s+(.+))?$', exercise_info)
            if exercise_match:
                exercise_number = int(exercise_match.group(1))
                exercise_name = exercise_match.group(2).strip()
                equipment = exercise_match.group(3).strip() if exercise_match.group(3) else None
                
                # Parse sets
                sets_match = re.search(r'(\d+)\s+sets?', goal_sets, re.IGNORECASE) if goal_sets else None
                goal_sets_num = int(sets_match.group(1)) if sets_match else None
                
                # Parse quantity and measure
                goal_quantity = None
                measure_quantity = None
                
                if goal_quantity_info:
                    # Check for reps
                    reps_match = re.search(r'(\d+)\s+reps?', goal_quantity_info, re.IGNORECASE)
                    if reps_match:
                        goal_quantity = int(reps_match.group(1))
                        measure_quantity = "reps"
                    else:
                        # Check for time (seconds)
                        time_match = re.search(r'(\d+)\s+sec', goal_quantity_info, re.IGNORECASE)
                        if time_match:
                            goal_quantity = int(time_match.group(1))
                            measure_quantity = "sec"
                        # If no specific measure found, it might be a hold or similar
                        elif goal_quantity_info and not reps_match:
                            measure_quantity = "hold"
                
                # Add to parsed data
                if current_program:
                    row = {
                        'plan_name': current_program['plan_name'],
                        'duration_weeks': current_program['duration_weeks'],
                        'start_date': current_program['start_date'],
                        'day_number': current_day_number,
                        'day_name': current_day,
                        'exercise_number': exercise_number,
                        'exercise_name': exercise_name,
                        'equipment': equipment,
                        'goal_sets': goal_sets_num,
                        'goal_quantity': goal_quantity,
                        'measure_quantity': measure_quantity
                    }
                    parsed_data.append(row)
    
    # Convert to DataFrame
    df = pd.DataFrame(parsed_data)
    
    return df

In [145]:
#parse_plans(PLANS_FILE)

#### Workouts

In [146]:
def parse_workouts_csv(file_path):
    # Read the raw file
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
    
    # Split by workout sessions (looking for workout headers - both regular and standalone)
    workout_blocks = re.split(r'(?=^"[^"]*(?:Day\s+\d+|Standalone workout)[^"]*")', content, flags=re.MULTILINE)
    workout_blocks = [block.strip() for block in workout_blocks if block.strip()]
    
    parsed_data = []
    
    for block in workout_blocks:
        lines = [line.strip() for line in block.split('\n') if line.strip()]
        if not lines:
            continue
            
        # Parse workout header
        header_line = lines[0]
        header_parts = [part.strip('"') for part in re.split(r'";"|^"|"$', header_line) if part]
        
        if len(header_parts) < 3:
            continue
            
        # Parse workout metadata
        workout_info = header_parts[0]
        dt_info = header_parts[1]
        duration_info = header_parts[2]
        
        # Extract workout components (handle both plan and standalone workouts)
        workout_parts = [part.strip() for part in workout_info.split('·')]
        
        if len(workout_parts) == 2 and 'standalone' in workout_parts[1].lower():
            # Standalone workout (either "Standalone workout · Standalone workout" or "PerGlu · Standalone workout")
            day_name = workout_parts[0]
            day_number = 0
            week_number = 0
            plan_name = "Standalone workout"
        elif len(workout_parts) >= 4:
            # Regular plan workout
            day_name = workout_parts[0]
            day_match = re.search(r'Day\s+(\d+)', workout_parts[1])
            day_number = int(day_match.group(1)) if day_match else None
            week_match = re.search(r'Week\s+(\d+)', workout_parts[2])
            week_number = int(week_match.group(1)) if week_match else None
            plan_name = workout_parts[3]
        else:
            continue
        
        # Parse datetime
        dt_match = re.search(r'(\d{4}-\d{2}-\d{2})\s+(\d{1,2}):(\d{2})', dt_info)
        if dt_match:
            date_str = dt_match.group(1)
            hour = dt_match.group(2)
            minute = dt_match.group(3)
            dt_start = datetime.strptime(f"{date_str} {hour}:{minute}", "%Y-%m-%d %H:%M")
        else:
            dt_start = None
            
        # Parse duration (handle both minutes and hours)
        duration_min = None
        # Check for hours format (e.g., "1:04 hr")
        hour_match = re.search(r'(\d+):(\d+)\s*hr', duration_info)
        if hour_match:
            hours = int(hour_match.group(1))
            minutes = int(hour_match.group(2))
            duration_min = hours * 60 + minutes
        else:
            # Check for minutes format (e.g., "59 min")
            min_match = re.search(r'(\d+)\s*min', duration_info)
            duration_min = int(min_match.group(1)) if min_match else None
        
        # Process exercises in this workout
        current_exercise = None
        current_notes = None
        set_data_mode = False
        header_columns = []
        
        for line in lines[1:]:
            # Check if this is an exercise header
            exercise_match = re.match(r'^"(\d+)\.\s*([^"]+)"', line)
            if exercise_match:
                exercise_number = int(exercise_match.group(1))
                exercise_info = exercise_match.group(2)
                
                # Parse exercise components
                exercise_parts = [part.strip() for part in exercise_info.split('·')]
                if len(exercise_parts) >= 3:
                    exercise_name = exercise_parts[0]
                    equipment = exercise_parts[1]
                    goal_info = exercise_parts[2]
                    
                    # Extract goal quantity and measure
                    goal_match = re.search(r'(\d+)\s+(reps?|sec)', goal_info, re.IGNORECASE)
                    if goal_match:
                        goal_quantity = int(goal_match.group(1))
                        measure_quantity = goal_match.group(2).lower()
                        if measure_quantity.startswith('rep'):
                            measure_quantity = 'reps'
                        elif measure_quantity == 'sec':
                            measure_quantity = 'secs'
                    else:
                        goal_quantity = None
                        measure_quantity = None
                    
                    current_exercise = {
                        'exercise_number': exercise_number,
                        'exercise_name': exercise_name,
                        'equipment': equipment,
                        'goal_quantity': goal_quantity,
                        'measure_quantity': measure_quantity
                    }
                
                # Check for notes in the same line or next line
                notes_match = re.search(r';"([^"]*)"', line)
                current_notes = notes_match.group(1) if notes_match else None
                set_data_mode = False
                continue
            
            # Check for notes line (if not in exercise header)
            if current_notes is None and line.startswith('"') and line.endswith('"'):
                current_notes = line.strip('"')
                continue
                
            # Check for set data header
            if line.startswith('#;'):
                header_columns = [col.strip() for col in line[2:].split(';')]
                set_data_mode = True
                continue
            
            # Parse set data
            if set_data_mode and current_exercise and re.match(r'^\d+;', line):
                set_parts = line.split(';')
                if len(set_parts) >= 2:
                    try:
                        set_number = int(set_parts[0])
                        
                        # Initialize values
                        weight = None
                        reps = None
                        duration = None
                        
                        # Parse based on header columns
                        for i, col_name in enumerate(header_columns):
                            if i + 1 < len(set_parts):  # +1 because set_parts[0] is set number
                                value_str = set_parts[i + 1].strip()
                                
                                if col_name.upper() == 'KG' and value_str:
                                    # Parse weight (handle +0 notation)
                                    if value_str == '+0':
                                        weight = 0.0
                                    elif value_str.startswith('+'):
                                        weight = float(value_str[1:].replace(',', '.'))
                                    else:
                                        weight = float(value_str.replace(',', '.'))
                                        
                                elif col_name.upper() == 'REPS' and value_str:
                                    reps = int(value_str)
                                    
                                elif col_name.upper() in ['SECS', 'SEC'] and value_str:
                                    duration = int(value_str)
                                    
                                elif col_name.upper() in ['MINS', 'MIN'] and value_str:
                                    # Parse time in MM:SS or just seconds
                                    time_match = re.search(r'(\d+):(\d+)', value_str)
                                    if time_match:
                                        minutes = int(time_match.group(1))
                                        seconds = int(time_match.group(2))
                                        duration = minutes * 60 + seconds
                                    else:
                                        # Assume it's just seconds
                                        duration = int(value_str)
                        
                        # Create row
                        row = {
                            'day_name': day_name,
                            'day_number': day_number,
                            'week_number': week_number,
                            'plan_name': plan_name,
                            'dt_start': dt_start,
                            'duration_min': duration_min,
                            'exercise_number': current_exercise['exercise_number'],
                            'exercise_name': current_exercise['exercise_name'],
                            'equipment': current_exercise['equipment'],
                            'goal_quantity': current_exercise['goal_quantity'],
                            'measure_quantity': current_exercise['measure_quantity'],
                            'notes': current_notes,
                            'set_number': set_number,
                            'weight': weight,
                            'reps': reps,
                            'duration': duration
                        }
                        parsed_data.append(row)
                        
                    except (ValueError, IndexError):
                        continue
    
    # Convert to DataFrame
    df = pd.DataFrame(parsed_data)
    
    return df

In [147]:
#parse_workouts_csv(WORKOUTS_FILE)

#### Get IDs

In [ ]:
def generate_exercise_id(exercise_name_column, equipment_column):
    df = pd.DataFrame()

    df['exercise_name_formatted'] = (
        exercise_name_column
        .str.lower()
        .str.replace("on ", "", regex=True)
        .str.replace(" ", "", regex=False)
        .str.replace("-", "", regex=False)
    )

    df['equipment_short'] = (
        equipment_column
        .map({
            'Bodyweight': 'BW',
            'Machine': 'MCH',
            'Cable': 'CAB',
            'Barbell': 'BB',
            'Dumbbells': 'DB',
            'Kettlebells': 'KB',
            'EZ bar': 'EZB',
            'Trap bar': 'TB',
            'TRX': 'TRX',
            'Smith machine': 'SMT',
            'Resistance bands': 'RES',
            })
        .fillna('MISSING')
    )

    return df['exercise_name_formatted'] + '_' + df['equipment_short']

In [175]:
def generate_plan_id(plan_name_column, start_date_column):
    # Remove non-alphanumeric characters and lowercase the plan name
    plan_name_clean = (
        plan_name_column
        .str.lower()
        .str.replace(r'[^a-zA-Z0-9]', '', regex=True)
    )
    # Format start date as YYYYMM
    start_date_fmt = pd.to_datetime(start_date_column).dt.strftime('%Y%m')
    # Combine to form plan_id
    return start_date_fmt + '_' + plan_name_clean

#### Others

In [7]:
def extract_day_or_week(value):
    if value == "Standalone workout":
        return 0
    elif value.split()[-1].isdigit():
        return int(value.split()[-1])

In [8]:
def convert_to_minutes(duration):
    if "hr" in duration:
        if ':' in duration:
            hours, minutes = duration.replace(" hr", "").split(":")
            return int(hours) * 60 + int(minutes)
        else:
            return int(duration.replace(" hr", "")) * 60
    elif "min" in duration:
        return int(duration.replace(" min", ""))
    else:
        return None

In [9]:
def convert_to_seconds(duration):
    if pd.isna(duration) or duration == '' or duration == 0:
        return 0
    elif ':' in duration:
        minutes, seconds = duration.split(":")
        return int(minutes) * 60 + int(seconds)
    else:
        return int(duration) * 60

In [10]:
def strip_measure(value):
    if (pd.isna(value) or value == ''):
        return 0
    else:
        return value.split()[0]

### Load

In [12]:
def insert_df(df, sql_table, schema='public', chunk_size=3000):
  num_chunks = (len(df) // chunk_size) + 1

  print(f"Inserting {len(df)} rows in {num_chunks} chunks.\n")

  colunas_df = list(df.columns)

  # Connection to the PostgreSQL database
  conn = psycopg2.connect(
      host=db_host,
      database=db_name,
      user=db_user,
      password=db_password
  )
  cur = conn.cursor()

  cur.execute(f"SET search_path TO {schema}")

  for i in range(num_chunks):
    start_idx = i * chunk_size
    end_idx = min((i + 1) * chunk_size, len(df))
    chunk_df = df.iloc[start_idx:end_idx]

    print(f"Inserting rows {start_idx}:{end_idx}")

    # tl == Tuple List
    tl_df = list(chunk_df.itertuples(index=False, name=None))

    insert_df = sql.SQL("INSERT INTO {} ({}) VALUES %s").format(
      sql.Identifier(sql_table),
      sql.SQL(', ').join(map(sql.Identifier, colunas_df))
    )

    # Execute the query with the data
    try:
        extras.execute_values(cur, insert_df, tl_df)
        conn.commit()
        print("Rows inserted successfuly.")
    except Exception as e:
        conn.rollback()
        print(f"Error: {e}")

  # Close the cursor and connection
  cur.close()
  conn.close()

  print("FINISHED")

  return None

# Dataframes - Additional Transformations

## Plans

In [153]:
df_plans_orig = parse_plans(PLANS_FILE)
df_plans_transf = df_plans_orig.copy()

In [188]:
df_plans_transf = df_plans_orig.assign(
    exercise_ref_id = generate_exercise_id(df_plans_orig['exercise_name'], df_plans_orig['equipment']),
    plan_id = generate_plan_id(df_plans_orig['plan_name'], df_plans_orig['start_date']),
    plan_exercise_id = lambda x: x['plan_id'] + '_' + x['exercise_number'].astype(str),
)

In [192]:
df_plans_transf = df_plans_transf[[
    'plan_exercise_id',
    'plan_id',
    'exercise_ref_id',
    'start_date',
    'plan_name',
    'duration_weeks',
    'day_number',
    'day_name',
    'exercise_number',
    'exercise_name',
    'equipment',
    'goal_sets',
    'goal_quantity',
    'measure_quantity'
]]

In [194]:
#df_plans_transf

## Exercises

### Custom Exercises

In [166]:
df_customexercises_orig = pd.read_csv(CUSTOMEXERCISES_FILE, skiprows=1, sep=';')
df_customexercises_transf = df_customexercises_orig.copy()

In [168]:
#df_customexercises_transf

### Standard Exercises

# Load

In [156]:
#df_workouts_formatted.to_csv(f"{GOLD_FOLDER}/workouts.csv", sep=";", decimal=',', index=False)
#df_exercisesref_final.to_csv(f"{GOLD_FOLDER}/exercises_ref.csv", sep=";", decimal=',', index=False)
#df_exerciseslog_formatted.to_csv(f"{GOLD_FOLDER}/exercises_log.csv", sep=";", decimal=',', index=False)

In [162]:
#insert_df(df_workouts_formatted, 'gold_dim_workouts', 'ap_workouts')

In [ ]:
#insert_df(df_exercisesref_final, 'gold_dim_exercisesref', 'ap_workouts')

In [ ]:
#insert_df(df_exerciseslog_formatted, 'gold_fact_sets', 'ap_workouts')

# Tests

In [70]:
#df_workouts_formatted

In [87]:
#df_exerciseslog_formatted

In [88]:
#df_exercises_ref

# Backups

In [ ]:
exercise_groups = [
{'exercise_name': 'Archer Push-Ups', 'equipment': 'Bodyweight', 'muscle_group': 'Upper Body', 'exercise_group': 'Push-Up'},
{'exercise_name': 'Bilateral Calf Press - Horizontal Leg Press', 'equipment': 'Machine', 'muscle_group': 'Lower Body', 'exercise_group': 'Calf Press'},
{'exercise_name': 'Chair Crunches', 'equipment': 'Bodyweight', 'muscle_group': 'Core', 'exercise_group': 'Ab Crunch'},
{'exercise_name': 'Decline Push-Ups', 'equipment': 'Bodyweight', 'muscle_group': 'Upper Body', 'exercise_group': 'Push-Up'},
{'exercise_name': 'Diagonal Hip Abduction', 'equipment': 'Machine', 'muscle_group': 'Lower Body', 'exercise_group': 'Hip Abduction'},
{'exercise_name': 'Elevated Pike hold', 'equipment': 'Bodyweight', 'muscle_group': 'Upper Body', 'exercise_group': 'Pike'},
{'exercise_name': 'Explosive Push-Ups', 'equipment': 'Bodyweight', 'muscle_group': 'Upper Body', 'exercise_group': 'Push-Up'},
{'exercise_name': 'Hanging Knee Tuck - iso', 'equipment': 'Bodyweight', 'muscle_group': 'Core', 'exercise_group': 'Leg Raise'},
{'exercise_name': 'Incline Archer Push-Ups', 'equipment': 'Bodyweight', 'muscle_group': 'Upper Body', 'exercise_group': 'Push-Up'},
{'exercise_name': 'Incline Australian Pull-Ups', 'equipment': 'Bodyweight', 'muscle_group': 'Upper Body', 'exercise_group': 'Row'},
{'exercise_name': 'Knee Diamond Push-Ups', 'equipment': 'Bodyweight', 'muscle_group': 'Upper Body', 'exercise_group': 'Push-Up'},
{'exercise_name': 'L Raises', 'equipment': 'Bodyweight', 'muscle_group': 'Core', 'exercise_group': 'Leg Raise'},
{'exercise_name': 'L-Sit Hold', 'equipment': 'Bodyweight', 'muscle_group': 'Core', 'exercise_group': 'Leg Raise'},
{'exercise_name': 'Pike Push-Ups', 'equipment': 'Bodyweight', 'muscle_group': 'Upper Body', 'exercise_group': 'Pike'},
{'exercise_name': 'Pseudo Planche Lean - Iso', 'equipment': 'Bodyweight', 'muscle_group': 'Upper Body', 'exercise_group': 'Push-Up'},
{'exercise_name': 'Pseudo Push-Ups', 'equipment': 'Bodyweight', 'muscle_group': 'Upper Body', 'exercise_group': 'Push-Up'},
{'exercise_name': 'Scapular Pull-Ups', 'equipment': 'Bodyweight', 'muscle_group': 'Upper Body', 'exercise_group': 'Pull-Up'},
{'exercise_name': 'Single Leg Hip Thrust', 'equipment': 'Bodyweight', 'muscle_group': 'Lower Body', 'exercise_group': 'Hip Thrust'},
{'exercise_name': 'Single leg deadlift', 'equipment': 'Bodyweight', 'muscle_group': 'Lower Body', 'exercise_group': 'Other'},
{'exercise_name': 'Skin the cat', 'equipment': 'Bodyweight', 'muscle_group': 'Upper Body', 'exercise_group': 'Lat Pull'},
{'exercise_name': 'Skydiver', 'equipment': 'Bodyweight', 'muscle_group': 'Core', 'exercise_group': 'Back Extension'},
{'exercise_name': 'Skydiver - Iso', 'equipment': 'Bodyweight', 'muscle_group': 'Core', 'exercise_group': 'Back Extension'},
{'exercise_name': 'Weighted L Raises', 'equipment': 'Bodyweight', 'muscle_group': 'Core', 'exercise_group': 'Leg Raise'},
{'exercise_name': 'Assisted Chin-Ups', 'equipment': 'Machine', 'muscle_group': 'Upper Body', 'exercise_group': 'Pull-Up'},
{'exercise_name': 'Assisted Dips', 'equipment': 'Machine', 'muscle_group': 'Upper Body', 'exercise_group': 'Dip'},
{'exercise_name': 'Back Extensions on Roman Chair', 'equipment': 'Bodyweight', 'muscle_group': 'Core', 'exercise_group': 'Back Extension'},
{'exercise_name': 'Bench Dips', 'equipment': 'Bodyweight', 'muscle_group': 'Upper Body', 'exercise_group': 'Dip'},
{'exercise_name': 'Bench Press', 'equipment': 'Barbell', 'muscle_group': 'Upper Body', 'exercise_group': 'Chest Press'},
{'exercise_name': 'Bench Press', 'equipment': 'Dumbbells', 'muscle_group': 'Upper Body', 'exercise_group': 'Chest Press'},
{'exercise_name': 'Bent-Over Rows', 'equipment': 'Barbell', 'muscle_group': 'Upper Body', 'exercise_group': 'Row'},
{'exercise_name': 'Bulgarian Split Squats', 'equipment': 'Dumbbells', 'muscle_group': 'Lower Body', 'exercise_group': 'Squat'},
{'exercise_name': 'Burpees', 'equipment': 'Bodyweight', 'muscle_group': 'Full Body', 'exercise_group': 'Cardio'},
{'exercise_name': 'Butterfly with Close Grip', 'equipment': 'Machine', 'muscle_group': 'Upper Body', 'exercise_group': 'Fly'},
{'exercise_name': 'Butterfly with Wide Grip', 'equipment': 'Machine', 'muscle_group': 'Upper Body', 'exercise_group': 'Fly'},
{'exercise_name': 'Calf Raises on Leg Press', 'equipment': 'Machine', 'muscle_group': 'Lower Body', 'exercise_group': 'Calf Press'},
{'exercise_name': 'Chest Press', 'equipment': 'Machine', 'muscle_group': 'Upper Body', 'exercise_group': 'Chest Press'},
{'exercise_name': 'Chest-Supported Low Rows with Neutral Grip', 'equipment': 'Machine', 'muscle_group': 'Upper Body', 'exercise_group': 'Row'},
{'exercise_name': 'Chest-Supported Rows with Wide Grip', 'equipment': 'Machine', 'muscle_group': 'Upper Body', 'exercise_group': 'Row'},
{'exercise_name': 'Concentration Curls', 'equipment': 'Dumbbells', 'muscle_group': 'Upper Body', 'exercise_group': 'Biceps Curl'},
{'exercise_name': 'Crunch Machine', 'equipment': 'Machine', 'muscle_group': 'Core', 'exercise_group': 'Ab Crunch'},
{'exercise_name': 'Curls', 'equipment': 'Barbell', 'muscle_group': 'Upper Body', 'exercise_group': 'Biceps Curl'},
{'exercise_name': 'Curls', 'equipment': 'Cable', 'muscle_group': 'Upper Body', 'exercise_group': 'Biceps Curl'},
{'exercise_name': 'Curls', 'equipment': 'Dumbbells', 'muscle_group': 'Upper Body', 'exercise_group': 'Biceps Curl'},
{'exercise_name': 'Curls', 'equipment': 'EZ bar', 'muscle_group': 'Upper Body', 'exercise_group': 'Biceps Curl'},
{'exercise_name': 'Decline Crunches', 'equipment': 'Bodyweight', 'muscle_group': 'Core', 'exercise_group': 'Ab Crunch'},
{'exercise_name': 'Dips', 'equipment': 'Bodyweight', 'muscle_group': 'Upper Body', 'exercise_group': 'Dip'},
{'exercise_name': 'Double Crunch Machine', 'equipment': 'Machine', 'muscle_group': 'Core', 'exercise_group': 'Ab Crunch'},
{'exercise_name': 'Dragon Flag', 'equipment': 'Bodyweight', 'muscle_group': 'Core', 'exercise_group': 'Other'},
{'exercise_name': 'Flys', 'equipment': 'Cable', 'muscle_group': 'Upper Body', 'exercise_group': 'Fly'},
{'exercise_name': 'Flys', 'equipment': 'Dumbbells', 'muscle_group': 'Upper Body', 'exercise_group': 'Fly'},
{'exercise_name': 'Forward Lunges', 'equipment': 'Dumbbells', 'muscle_group': 'Lower Body', 'exercise_group': 'Lunge'},
{'exercise_name': 'Front Raises', 'equipment': 'Barbell', 'muscle_group': 'Upper Body', 'exercise_group': 'Front Raise'},
{'exercise_name': 'Front Raises', 'equipment': 'Dumbbells', 'muscle_group': 'Upper Body', 'exercise_group': 'Front Raise'},
{'exercise_name': 'Goblet Squats', 'equipment': 'Dumbbells', 'muscle_group': 'Lower Body', 'exercise_group': 'Squat'},
{'exercise_name': 'Hanging Leg Raises', 'equipment': 'Bodyweight', 'muscle_group': 'Core', 'exercise_group': 'Leg Raise'},
{'exercise_name': 'Hip Abduction', 'equipment': 'Cable', 'muscle_group': 'Lower Body', 'exercise_group': 'Hip Abduction'},
{'exercise_name': 'Hip Abduction', 'equipment': 'Machine', 'muscle_group': 'Lower Body', 'exercise_group': 'Hip Abduction'},
{'exercise_name': 'Hip Adduction', 'equipment': 'Cable', 'muscle_group': 'Lower Body', 'exercise_group': 'Hip Abduction'},
{'exercise_name': 'Hip Adduction', 'equipment': 'Machine', 'muscle_group': 'Lower Body', 'exercise_group': 'Hip Abduction'},
{'exercise_name': 'Hip Thrusts', 'equipment': 'Barbell', 'muscle_group': 'Lower Body', 'exercise_group': 'Hip Thrust'},
{'exercise_name': 'Incline Bench Press', 'equipment': 'Barbell', 'muscle_group': 'Upper Body', 'exercise_group': 'Chest Press'},
{'exercise_name': 'Incline Bench Press', 'equipment': 'Dumbbells', 'muscle_group': 'Upper Body', 'exercise_group': 'Chest Press'},
{'exercise_name': 'Incline Reverse Flys', 'equipment': 'Dumbbells', 'muscle_group': 'Upper Body', 'exercise_group': 'Reverse Fly'},
{'exercise_name': 'Incline Rows', 'equipment': 'Dumbbells', 'muscle_group': 'Upper Body', 'exercise_group': 'Row'},
{'exercise_name': 'Inverted Rows on Smith Machine', 'equipment': 'Bodyweight', 'muscle_group': 'Upper Body', 'exercise_group': 'Row'},
{'exercise_name': 'Inverted Rows on Smith Machine with Reverse Grip', 'equipment': 'Bodyweight', 'muscle_group': 'Upper Body', 'exercise_group': 'Row'},
{'exercise_name': 'Kneeling Crunches', 'equipment': 'Cable', 'muscle_group': 'Core', 'exercise_group': 'Ab Crunch'},
{'exercise_name': 'Lat Pulldowns with Close Neutral Grip', 'equipment': 'Cable', 'muscle_group': 'Upper Body', 'exercise_group': 'Pulldown'},
{'exercise_name': 'Lat Pulldowns with Wide Neutral Grip', 'equipment': 'Cable', 'muscle_group': 'Upper Body', 'exercise_group': 'Pulldown'},
{'exercise_name': 'Lat Pulldowns with Wide Overhand Grip', 'equipment': 'Cable', 'muscle_group': 'Upper Body', 'exercise_group': 'Pulldown'},
{'exercise_name': 'Lateral Raises', 'equipment': 'Cable', 'muscle_group': 'Upper Body', 'exercise_group': 'Lateral Raise'},
{'exercise_name': 'Lateral Raises', 'equipment': 'Dumbbells', 'muscle_group': 'Upper Body', 'exercise_group': 'Lateral Raise'},
{'exercise_name': 'Lean Away Lateral Raises', 'equipment': 'Cable', 'muscle_group': 'Upper Body', 'exercise_group': 'Lateral Raise'},
{'exercise_name': 'Leg Curls on Leg Extension Machine', 'equipment': 'Machine', 'muscle_group': 'Lower Body', 'exercise_group': 'Leg Curl'},
{'exercise_name': 'Leg Extensions', 'equipment': 'Machine', 'muscle_group': 'Lower Body', 'exercise_group': 'Leg Extension'},
{'exercise_name': 'Leg Press', 'equipment': 'Machine', 'muscle_group': 'Lower Body', 'exercise_group': 'Leg Press'},
{'exercise_name': 'Leg Raises', 'equipment': 'Bodyweight', 'muscle_group': 'Core', 'exercise_group': 'Leg Raise'},
{'exercise_name': 'Lying Leg Curls', 'equipment': 'Machine', 'muscle_group': 'Lower Body', 'exercise_group': 'Leg Curl'},
{'exercise_name': 'One-Arm Rows', 'equipment': 'Dumbbells', 'muscle_group': 'Upper Body', 'exercise_group': 'Row'},
{'exercise_name': 'Overhead Triceps Extensions', 'equipment': 'Dumbbells', 'muscle_group': 'Upper Body', 'exercise_group': 'Triceps Extension'},
{'exercise_name': 'Pistol Squats', 'equipment': 'TRX', 'muscle_group': 'Lower Body', 'exercise_group': 'Squat'},
{'exercise_name': 'Plank', 'equipment': 'Bodyweight', 'muscle_group': 'Core', 'exercise_group': 'Plank'},
{'exercise_name': 'Preacher Curls', 'equipment': 'Machine', 'muscle_group': 'Upper Body', 'exercise_group': 'Biceps Curl'},
{'exercise_name': 'Pull-Ups with Close Neutral Grip', 'equipment': 'Bodyweight', 'muscle_group': 'Upper Body', 'exercise_group': 'Pull-Up'},
{'exercise_name': 'Pull-Ups with Wide Overhand Grip', 'equipment': 'Bodyweight', 'muscle_group': 'Upper Body', 'exercise_group': 'Pull-Up'},
{'exercise_name': 'Pullovers', 'equipment': 'Cable', 'muscle_group': 'Upper Body', 'exercise_group': 'Lat Pull'},
{'exercise_name': 'Push-Ups', 'equipment': 'Bodyweight', 'muscle_group': 'Upper Body', 'exercise_group': 'Push-Up'},
{'exercise_name': 'Reverse Butterfly with Wide Grip', 'equipment': 'Machine', 'muscle_group': 'Upper Body', 'exercise_group': 'Reverse Fly'},
{'exercise_name': 'Romanian Deadlifts', 'equipment': 'Barbell', 'muscle_group': 'Lower Body', 'exercise_group': 'Deadlift'},
{'exercise_name': 'Rows with Close Grip', 'equipment': 'Cable', 'muscle_group': 'Upper Body', 'exercise_group': 'Row'},
{'exercise_name': 'Rows with Close Grip', 'equipment': 'TRX', 'muscle_group': 'Upper Body', 'exercise_group': 'Row'},
{'exercise_name': 'Rows with Reverse Grip', 'equipment': 'Cable', 'muscle_group': 'Upper Body', 'exercise_group': 'Row'},
{'exercise_name': 'Rows with Wide Neutral Grip', 'equipment': 'Cable', 'muscle_group': 'Upper Body', 'exercise_group': 'Row'},
{'exercise_name': 'Russian Swings', 'equipment': 'Kettlebells', 'muscle_group': 'Upper Body', 'exercise_group': 'Row'},
{'exercise_name': 'Russian Twists', 'equipment': 'Bodyweight', 'muscle_group': 'Core', 'exercise_group': 'Other'},
{'exercise_name': 'Seated Calf Raises', 'equipment': 'Machine', 'muscle_group': 'Lower Body', 'exercise_group': 'Calf Press'},
{'exercise_name': 'Seated Curls', 'equipment': 'Dumbbells', 'muscle_group': 'Upper Body', 'exercise_group': 'Biceps Curl'},
{'exercise_name': 'Seated Lateral Raises', 'equipment': 'Dumbbells', 'muscle_group': 'Upper Body', 'exercise_group': 'Lateral Raise'},
{'exercise_name': 'Seated Leg Curls', 'equipment': 'Machine', 'muscle_group': 'Lower Body', 'exercise_group': 'Leg Curl'},
{'exercise_name': 'Seated Shoulder Press', 'equipment': 'Barbell', 'muscle_group': 'Upper Body', 'exercise_group': 'Shoulder Press'},
{'exercise_name': 'Seated Shoulder Press', 'equipment': 'Dumbbells', 'muscle_group': 'Upper Body', 'exercise_group': 'Shoulder Press'},
{'exercise_name': 'Shoulder Press', 'equipment': 'Machine', 'muscle_group': 'Upper Body', 'exercise_group': 'Shoulder Press'},
{'exercise_name': 'Side Bends on Roman Chair', 'equipment': 'Bodyweight', 'muscle_group': 'Core', 'exercise_group': 'Side Abs'},
{'exercise_name': 'Side Plank', 'equipment': 'Bodyweight', 'muscle_group': 'Core', 'exercise_group': 'Side Abs'},
{'exercise_name': 'Side Split Squats', 'equipment': 'Dumbbells', 'muscle_group': 'Lower Body', 'exercise_group': 'Squat'},
{'exercise_name': 'Skull Crushers', 'equipment': 'Cable', 'muscle_group': 'Upper Body', 'exercise_group': 'Triceps Extension'},
{'exercise_name': 'Skull Crushers', 'equipment': 'EZ bar', 'muscle_group': 'Upper Body', 'exercise_group': 'Triceps Extension'},
{'exercise_name': 'Squats', 'equipment': 'Barbell', 'muscle_group': 'Lower Body', 'exercise_group': 'Squat'},
{'exercise_name': 'Squats', 'equipment': 'Smith machine', 'muscle_group': 'Lower Body', 'exercise_group': 'Squat'},
{'exercise_name': 'Standing Calf Raises', 'equipment': 'Bodyweight', 'muscle_group': 'Lower Body', 'exercise_group': 'Calf Press'},
{'exercise_name': 'Standing Calf Raises', 'equipment': 'Machine', 'muscle_group': 'Lower Body', 'exercise_group': 'Calf Press'},
{'exercise_name': 'Standing Shoulder Press', 'equipment': 'Barbell', 'muscle_group': 'Upper Body', 'exercise_group': 'Shoulder Press'},
{'exercise_name': 'Triceps Pushdowns', 'equipment': 'Cable', 'muscle_group': 'Upper Body', 'exercise_group': 'Triceps Extension'},
{'exercise_name': 'Triceps Pushdowns with Rope', 'equipment': 'Cable', 'muscle_group': 'Upper Body', 'exercise_group': 'Triceps Extension'},
{'exercise_name': 'Upright Rows', 'equipment': 'Barbell', 'muscle_group': 'Upper Body', 'exercise_group': 'Row'},
{'exercise_name': 'Elevated Dragon Squats', 'equipment': 'Bodyweight', 'muscle_group': 'Lower Body', 'exercise_group': 'Squat'},
{'exercise_name': 'Frog stand hold', 'equipment': 'Bodyweight', 'muscle_group': 'Upper Body', 'exercise_group': 'Front Raise'},
{'exercise_name': 'Full Nordic Curls', 'equipment': 'Bodyweight', 'muscle_group': 'Lower Body', 'exercise_group': 'Leg Curl'},
{'exercise_name': 'Natural Leg Extensions', 'equipment': 'Bodyweight', 'muscle_group': 'Lower Body', 'exercise_group': 'Leg Extension'},
{'exercise_name': 'Negative Nordic Curls', 'equipment': 'Bodyweight', 'muscle_group': 'Lower Body', 'exercise_group': 'Leg Curl'},
{'exercise_name': 'Shrimp Squats', 'equipment': 'Bodyweight', 'muscle_group': 'Lower Body', 'exercise_group': 'Squat'},
{'exercise_name': 'Sumo Belt Squats', 'equipment': 'Machine', 'muscle_group': 'Lower Body', 'exercise_group': 'Squat'},
{'exercise_name': 'Typewriter pullups', 'equipment': 'Bodyweight', 'muscle_group': 'Upper Body', 'exercise_group': 'Pull-Up'},
{'exercise_name': 'Wall Handstand Hold', 'equipment': 'Bodyweight', 'muscle_group': 'Upper Body', 'exercise_group': 'Pike'},
{'exercise_name': 'Chest Press', 'equipment': 'TRX', 'muscle_group': 'Upper Body', 'exercise_group': 'Chest Press'},
{'exercise_name': 'Curls with Raised Arms', 'equipment': 'TRX', 'muscle_group': 'Upper Body', 'exercise_group': 'Biceps Curl'},
{'exercise_name': 'Flys', 'equipment': 'TRX', 'muscle_group': 'Upper Body', 'exercise_group': 'Fly'},
{'exercise_name': 'One-Arm Rows', 'equipment': 'TRX', 'muscle_group': 'Upper Body', 'exercise_group': 'Row'},
{'exercise_name': 'Pull-Ups with Wide Neutral Grip', 'equipment': 'Bodyweight', 'muscle_group': 'Upper Body', 'exercise_group': 'Pull-Up'},
{'exercise_name': 'Push-Ups with Wide Grip', 'equipment': 'Bodyweight', 'muscle_group': 'Upper Body', 'exercise_group': 'Push-Up'},
{'exercise_name': 'Reverse Flys', 'equipment': 'TRX', 'muscle_group': 'Upper Body', 'exercise_group': 'Reverse Fly'},
{'exercise_name': '', 'equipment': '', 'muscle_group': '', 'exercise_group': ''},
]